### Step 1 - Start Spark Session, include additional configurations and common functions, and read data from the spark-warehouse database

In [2]:
%run "../includes/configurations"

In [3]:
%run "../includes/common_functions"

In [4]:
from pyspark.sql.functions import concat, lit, to_timestamp

appName = "Races Ingestion"
spark = create_spark_context(appName, warehouse_location)

24/01/04 18:40:24 WARN Utils: Your hostname, falcao-sys resolves to a loopback address: 127.0.1.1; using 192.168.11.185 instead (on interface wlx7898e8c12476)
24/01/04 18:40:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/04 18:40:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/04 18:40:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/01/04 18:40:25 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
races_df = spark.read.parquet(f"{warehouse_location}/f1_raw.db/races")

### Step 2 - Rename and drop columns, and add new columns

In [6]:
races_column_mapping = {
    "raceId": "race_id",
    "year": "race_year",
    "circuitId": "circuit_id"
}

races_df = rename_columns(races_df, races_column_mapping)

In [7]:
columns_to_drop = ['url']

races_df = drop_columns(races_df, columns_to_drop)

In [8]:
races_df = add_ingestion_date(races_df)
races_df = races_df.withColumn(
    "race_timestemp", 
    to_timestamp(
        concat(
            races_df["date"],
            lit(" "),
            races_df["time"]
        ),
        "yyyy-MM-dd HH:mm:ss"
    )
)

### Step 3 - Write a data to a Datalake as a Parquet

In [9]:
races_df.write.mode("overwrite").parquet(f"{processed_folder_path}/races")